<a href="https://colab.research.google.com/github/Ajay07pandey/Interactive_question_answering_model/blob/main/Question_and_answer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Question_Answering_Model**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Self Projects/Question and answer Model /Data_set/S08_question_answer_pairs.txt', delimiter='\t')
df2 = pd.read_csv('/content/drive/MyDrive/Self Projects/Question and answer Model /Data_set/S09_question_answer_pairs.txt', delimiter = '\t')
df3 = pd.read_csv('/content/drive/MyDrive/Self Projects/Question and answer Model /Data_set/S10_question_answer_pairs.txt', delimiter = '\t', encoding = 'ISO-8859-1')

### 1. Merge all the data sets to one

In [ ]:
df = df1.append([df2,df3])

<ipython-input-9-8ef22378619a>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append([df2,df3])


In [ ]:
# First view of the data set
df.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,S08_set3_a4
4,Abraham_Lincoln,Did his mother die of pneumonia?,no,easy,medium,S08_set3_a4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3998 entries, 0 to 1457
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ArticleTitle              3998 non-null   object
 1   Question                  3961 non-null   object
 2   Answer                    3422 non-null   object
 3   DifficultyFromQuestioner  3043 non-null   object
 4   DifficultyFromAnswerer    3418 non-null   object
 5   ArticleFile               3996 non-null   object
dtypes: object(6)
memory usage: 218.6+ KB


### 2. Removing duplicates

In [ ]:
# using shape attribute
df.shape

(3998, 6)

In [ ]:
df.duplicated().sum()

309

In [ ]:
# Drop using drop_duplicates method
df = df.drop_duplicates()

### 3. Dropping Na values

In [ ]:
df = df.dropna()

In [ ]:
df.shape

(2507, 6)

### 4. Removing the duplicate questions

In [ ]:
df = df.drop_duplicates(subset='Question')

In [ ]:
df.shape

(1525, 6)

#### 5. Lower Casing

In [ ]:
# Lower Casing
df['Question'] = df['Question'].str.lower()
df['Answer'] = df['Answer'].str.lower()

For simplicity we only take question and answer

In [ ]:
new_df = df[['Question','Answer']]

In [ ]:
new_df

,Question,Answer
0,was abraham lincoln the sixteenth president of...,yes
2,did lincoln sign the national banking act of 1...,yes
4,did his mother die of pneumonia?,no
6,how many long was lincoln's formal education?,18 months
8,when did lincoln begin his political career?,1832
...,...,...
1448,what do zebras eat?,mainly grass
1450,what are zebras hunted for?,mainly for their skins
1452,what areas do the grevy's zebras inhabit?,semi-arid grasslands of ethiopia and northern ...
1454,which species of zebra is known as the common ...,"plains zebra (equus quagga, formerly equus bur..."


#### 6. Removing Punctuations

In [ ]:
# function to remove punctuations
def remove_punctuations(text):
    '''This function is used to remove the punctuations from the given sentence'''
    #imorting needed library
    import string
    # replacing the punctuations with no space, which in effect deletes the punctuation marks.
    translator = str.maketrans('', '', string.punctuation)
    # return the text stripped off punctuation marks
    return text.translate(translator)

In [ ]:
# Removing Punctuations from the content_detail
new_df['Question']= new_df['Question'].apply(remove_punctuations)

# Checking the observation after manipulation
new_df.iloc[281,]['Question']

<ipython-input-77-539dd46acae6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Question']= new_df['Question'].apply(remove_punctuations)


'where was james monroe born'

## Importing necessary libraries for NLP

In [ ]:
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from collections import OrderedDict

#### 7. Removing Stopwords & Removing White spaces

In [ ]:
# Downloading stopwords
nltk.download('stopwords')

# create a set of English stop words
stop_words = stopwords.words('english')

# displaying stopwords
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### 8. Tokenization

In [ ]:
# Downloading needed libraries
nltk.download('punkt')

# Tokenization
new_df['Question']= new_df['Question'].apply(nltk.word_tokenize)

# Checking the observation after manipulation
new_df.iloc[281,]['Question']

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-80-bc8a7c4149cb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Question']= new_df['Question'].apply(nltk.word_tokenize)


['where', 'was', 'james', 'monroe', 'born']

#### 9. Text Normalization

In [ ]:
# Normalizing Text (i.e., Stemming, Lemmatization etc.)
# Importing WordNetLemmatizer from nltk module
from nltk.stem import WordNetLemmatizer

# Creating instance for wordnet
wordnet  = WordNetLemmatizer()

In [ ]:
def lemmatizing_sentence(text):
    '''This function is used for lemmatizing (changing the given word into meaningfull word) the words from the given sentence'''
    text = [wordnet.lemmatize(word) for word in text]

    # joining the list of words with space separator
    text=  " ".join(text)

    # return the manipulated string
    return text

In [ ]:
# Downloading needed libraries
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Rephrasing text by applying defined lemmatizing function
new_df['Question']= new_df['Question'].apply(lemmatizing_sentence)

# Checking the observation after manipulation
df.iloc[281,]['Question']

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-83-13c46e9ffeb8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Question']= new_df['Question'].apply(lemmatizing_sentence)


'where wa james monroe born'

#### 10. Text Vectorization

In [ ]:
# Vectorizing Text
# Importing needed libraries
from sklearn.feature_extraction.text import TfidfVectorizer

# Creating instance
tfidfv = TfidfVectorizer()

In [ ]:
# Fit and transform the corpus
tfidf_matrix = tfidfv.fit_transform(new_df['Question'])

# Checking shape of the formed document matrix
print(tfidf_matrix.shape)

(1525, 2541)


In [ ]:
# Get feature names (words) and create a DataFrame for better visualization
feature_names = tfidfv.get_feature_names_out()
tfidf_df = pd.DataFrame(data=tfidf_matrix.toarray(), columns=feature_names)

print(tfidf_df)

      10000  101  1110  1251  1255  12th  156  1658  1763  1765  ...  \
0       0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0   0.0  ...   
1       0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0   0.0  ...   
2       0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0   0.0  ...   
3       0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0   0.0  ...   
4       0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0   0.0  ...   
...     ...  ...   ...   ...   ...   ...  ...   ...   ...   ...  ...   
1520    0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0   0.0  ...   
1521    0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0   0.0  ...   
1522    0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0   0.0  ...   
1523    0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0   0.0  ...   
1524    0.0  0.0   0.0   0.0   0.0   0.0  0.0   0.0   0.0   0.0  ...   

         zebra  zholes  zhonghua  zhou  zhoukoudian  zihai  zone  zoo  zulu  \
0     0.000000     0.0       0.0   0.0          0.0    0

In [ ]:
tfidf_df.shape

(1525, 2541)

### Model using cosine similarity

In [ ]:
def general(question):
    query_vect = tfidfv.transform([question])
    similarity = cosine_similarity(query_vect, tfidf_matrix)
    index_similarity = np.argmax(similarity, axis=None)

    return similarity, index_similarity

In [ ]:
def ask(question):
    print( question)

    similarity, index_similarity = general(question)
    answer = new_df.iloc[index_similarity]['Answer']

    if ((similarity[0, index_similarity])>0):
        print('Closest question found:', df.iloc[index_similarity]['Question'])
        print('Similarity: {:.2%}'.format(similarity[0, index_similarity]))
        print('Answer:', new_df.iloc[index_similarity]['Answer'])
    else:
        print("Not really any great matches for that, sorry")

In [ ]:
ask('What is the national language in Malaysia?')

What is the national language in Malaysia?
Closest question found: what is the national language in malaysia
Similarity: 100.00%
Answer: malay 


In [ ]:
ask('national Malaysia?')

national Malaysia?
Closest question found: what is the national language in malaysia
Similarity: 80.75%
Answer: malay 


In [ ]:
ask('Do worker ants have wings?')

Do worker ants have wings?
Closest question found: do worker ant have wing
Similarity: 71.12%
Answer: no, worker ants do not have wings.


In [ ]:
ask('What does the "voice" of a violin depend upon?')

What does the "voice" of a violin depend upon?
Closest question found: what doe the voice of a violin depend upon
Similarity: 97.19%
Answer: shape


### Model Using Levenshtein distance

In [ ]:
!pip install Levenshtein

### Python Threads

In [ ]:
import concurrent.futures
from Levenshtein import distance  # Using the Levenshtein distance function from the Levenshtein package
def similar_question_thread(input_str, dataset):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # Use list comprehension to create a list of (question, distance) tuples
        distances = [(question, executor.submit(calculate_levenshtein_distance, input_str, question)) for question in new_df['Question']]

        # Wait for all threads to complete and get the results
        results = [(question, future.result()) for question, future in distances]

        # Find the question with the minimum distance
        most_similar_question = min(results, key=lambda x: x[1])[0]

    return most_similar_question

In [ ]:
def levenshtein_distance(str1, str2):
    if len(str1) > len(str2):
        str1, str2 = str2, str1

    distances = range(len(str1) + 1)
    for index2, char2 in enumerate(str2):
        new_distances = [index2 + 1]
        for index1, char1 in enumerate(str1):
            if char1 == char2:
                new_distances.append(distances[index1])
            else:
                new_distances.append(1 + min((distances[index1], distances[index1 + 1], new_distances[-1])))
        distances = new_distances

    return f'levenshtein distance is: {distances[-1]}'

In [ ]:
input_string = 'What does the of a violin depend upon?'
dataset = new_df['Question']

In [ ]:
most_similar_question = similar_question_thread(input_string, dataset)
print("Most Similar Question:", most_similar_question)

Most Similar Question: what doe the voice of a violin depend upon


In [ ]:
str1 = 'What does the of a violin depend upon?'
str2 = most_similar_question

In [ ]:
levenshtein_distance(str1, str2)

'levenshtein distance is: 9'

**Parallelize with Python Threads**

## **Xg boost**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Assuming your dataset has 'question' as a feature and 'answer' as the label
X = new_df['Question']
y = new_df['Answer']

# Step 2: Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Convert text data to feature vectors (you might need to use techniques like TF-IDF or word embeddings)
# For simplicity, let's assume you're using TF-IDF here
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Step 4: Train an XGBoost model
params = {
    'objective': 'multi:softmax',  # Use 'binary:logistic' for binary classification
    'num_class': len(new_df['Answer'].unique()),  # Number of unique answers
    'eval_metric': 'merror',  # Use 'logloss' for binary classification
}



num_round = 100
dtrain = xgb.DMatrix(X_train_tfidf, label=y_train)
bst = xgb.train(params, dtrain, num_round)

# Step 5: Make predictions on the test set
dtest = xgb.DMatrix(X_test_tfidf)
y_pred = bst.predict(dtest)

# Step 6: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

TypeError: ignored

In [ ]:
# Step 7: Make predictions on a new input string
new_input = 'What does the voice of a violin depend upon?'
new_input_tfidf = tfidf_vectorizer.transform([new_input])
new_input_dmatrix = xgb.DMatrix(new_input_tfidf)
prediction = int(bst.predict(new_input_dmatrix)[0])

# Print the predicted answer
predicted_answer = df['Answer'].unique()[prediction]
print(f'Predicted Answer: {predicted_answer}')

Predicted Answer: he never made a public declaration of his private faith
